<a href="https://colab.research.google.com/github/RDGopal/IB9AU-2026/blob/main/RAG_2_Long_Context_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Setup Environment

This first section handles the installation of necessary libraries and imports modules required for our application. We'll install `gradio` to build our web interface and import `google.generativeai` to interact with the Gemini API.
```

In [ ]:
# 1. Install Gradio
!pip install -q gradio

import gradio as gr
import google.generativeai as genai
import time
import os
from google.colab import userdata

## 2. Configure Gemini API

Before using Google's Generative AI models, we need to set up the API key. This cell retrieves your `GEMINI_API_KEY` from Colab's user data secrets and configures the `genai` library. Make sure you have your API key stored as 'GEMINI_API_KEY' in the Colab secrets manager.
```

In [ ]:
# 1. Setup (Use the standard library)
# Get your API key from https://aistudio.google.com/

from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

## 3. Core Application Logic (Functions)

This section defines the main functions that power our RAG (Retrieval-Augmented Generation) application:

*   **`active_gemini_files`**: A global list to store file objects uploaded to the Gemini API.
*   **`upload_and_process_files(files)`**: This function takes file paths from the Gradio interface, uploads them to the Gemini API, monitors their processing status, and returns a log of the results.
*   **`chat_response(message, history)`**: This is the heart of the chat. It receives the user's message and chat history, constructs a prompt (including an 'auditor' system prompt to enforce citation from provided documents), and uses the `gemini-flash-latest` model to generate a response. It ensures that all claims are backed by source blocks.

In [ ]:
# Global list to store the uploaded file objects from Gemini
active_gemini_files = []

# --- FUNCTIONS ---

def upload_and_process_files(files):
    """
    Takes a list of file paths (from Gradio), uploads them to Gemini,
    waits for processing, and returns a status message.
    """
    global active_gemini_files
    active_gemini_files = [] # Clear previous files

    if not files:
        return "No files uploaded."

    status_log = ""

    for file_path in files:
        # Upload to Gemini
        print(f"Uploading {file_path}...")
        g_file = genai.upload_file(file_path)

        # Wait for processing
        while g_file.state.name == "PROCESSING":
            time.sleep(1)
            g_file = genai.get_file(g_file.name)

        if g_file.state.name == "FAILED":
            status_log += f"❌ Failed to process: {file_path}\n"
        else:
            active_gemini_files.append(g_file)
            status_log += f"✅ Ready: {g_file.display_name}\n"

    return status_log

def chat_response(message, history):
    """
    The chat function.
    - message: The user's current question.
    - history: Previous chat context (Gradio handles this).
    """
    if not active_gemini_files:
        return "⚠️ Please upload and process documents first."

    # The Auditor Prompt (Enforcing Citations)
    auditor_prompt = """
    You are a strict Compliance Auditor.
    Answer the user's question using ONLY the provided documents.

    CRITICAL INSTRUCTION:
    For every claim you make, you must provide a "Source Block" in this exact format:
    [Source: Document Name | Page/Section | Exact Quote]

    If the answer is not in the documents, state "Information not found in provided records."
    """

    # We combine the System Prompt + User Message + All Active Files
    # Note: We pass the *list* of file objects directly to the model
    input_content = [auditor_prompt, message] + active_gemini_files

    try:
        # Using the Flash model for large context window
        model = genai.GenerativeModel("gemini-flash-latest")
        response = model.generate_content(input_content)
        return response.text
    except Exception as e:
        return f"Error generating response: {str(e)}"

## 4. Build the Web Interface with Gradio

Here, we construct the user interface using the Gradio library. `gr.Blocks` allows for more flexible layouts. We create:

*   A **File Uploader**: For users to select and upload documents.
*   An **Upload Button**: To trigger the `upload_and_process_files` function.
*   A **Status Output**: To display the processing status of the uploaded files.
*   A **Chat Interface**: To interact with the AI model, including example questions.

The `upload_btn.click` line links the button click event to our `upload_and_process_files` function, passing the uploaded files as input and updating the status output.


In [ ]:
# --- BUILD THE WEB UI ---

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🏦 HR RAG")
    gr.Markdown("Upload any number of documents (PDFs, Images, CSVs) and ask questions. The AI will cite its sources.")

    with gr.Row():
        with gr.Column(scale=1):
            # File Uploader
            file_input = gr.File(
                label="1. Upload Documents",
                file_count="multiple",
                file_types=[".pdf", ".csv", ".txt", ".png", ".jpg"]
            )
            upload_btn = gr.Button("Process Files", variant="primary")
            status_output = gr.Textbox(label="Status", interactive=False, lines=4)

        with gr.Column(scale=2):
            # Chat Interface
            chatbot = gr.ChatInterface(
                fn=chat_response,
                examples=["What is the dental deductible?", "Who approved the FMLA request?", "Summarize the drug testing policy."],
                title="2. Chat"
            )

    # Link the Upload Button to the processing function
    upload_btn.click(
        fn=upload_and_process_files,
        inputs=file_input,
        outputs=status_output
    )


## 5. Launch the Application

Finally, this cell launches the Gradio web application. The `demo.launch(debug=True)` command starts the server, making the interface accessible via a public URL provided by Gradio. `debug=True` is useful for seeing detailed logs during development.

In [ ]:
# Launch the app
demo.launch(debug=True)